In [ ]:
!pip install wbdata
!pip install pandas
!pip install datetime
!pip install bayesian-optimization==1.3.0
!pip install scikit-optimize

In [ ]:
import wbdata
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import joblib


from collections import Counter
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix,
                             roc_auc_score, roc_curve, f1_score, recall_score,
                             precision_score, accuracy_score, precision_recall_curve)
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import cross_val_score, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV


***Préparation des données***

In [ ]:
pays = ["AF", "BD", "BJ", "BT", "BF", "BI", "KH", "CM", "TD", "KM", "CI",
                     "DJ", "DM", "ET", "GH", "GD", "KE", "MG", "MW", "ML", "MR",
                     "MZ", "NP", "NE", "RW", "SN", "TG", "UG", "ZM", "ZW","CF", "CG",
        "CD", "GN", "GW", "GY", "HT", "HN", "KI", "KG"]

start_date = datetime.datetime(2008, 1, 1)
end_date = datetime.datetime(2025, 1, 1)

indicators = {
    'DT.DOD.DECT.GN.ZS': 'External_Debt_GDP',
    'DT.TDS.DECT.EX.ZS': 'Debt_Service_Exports',
    'NY.GDP.MKTP.KD.ZG': 'GDP_Growth',
    'BN.CAB.XOKA.GD.ZS': 'Current_Account',
    'PA.NUS.FCRF': 'Exchange_Rate',
    'IQ.CPA.TRAN.XQ': 'Corruption_Governance',
    'DT.ODA.ODAT.GN.ZS': 'Net_ODA_received'
}

data = wbdata.get_dataframe(indicators,
                            country=pays,
                            date=(start_date, end_date))

data.reset_index(inplace=True)
data["Year"] = pd.DatetimeIndex(data["date"]).year
data["country_clean"] = data["country"].str.lower().str.strip()

labels_df = pd.read_excel('/content/DEBT_DATA.xlsx')
labels_df["Country_clean"] = labels_df["Country"].str.lower().str.strip()
labels_df["Year"] = labels_df["Year"].astype(int)

data_merged = pd.merge(
    data,
    labels_df,
    left_on=["country_clean", "Year"],
    right_on=["Country_clean", "Year"],
    how="left"
)

data_merged = data_merged.drop(columns=['Country_clean', 'Country', 'date', 'country_clean'])
data_merged = data_merged.rename(columns={'country': 'Country'})

cols = ['Country', 'Year', 'Risk'] + \
       [col for col in data_merged.columns if col not in ['Country', 'Year', 'Risk']]
data_merged = data_merged[cols]

print("Lignes avant merge :", len(data))
print("Lignes après merge :", len(data_merged))
print("Taux de match labels :", data_merged["Risk"].notna().mean())

data_merged = data_merged.dropna(subset=['Risk']).copy()

data_merged.head(150)
data_merged.shape
data_merged.isna().sum()

print(len(data_merged))

Lignes avant merge : 680
Lignes après merge : 680
Taux de match labels : 0.9882352941176471
672
